In [ ]:
# Installation of firedrake
try:
  import firedrake
except ImportError:
  !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
  import firedrak


# # Installation of libraries for mesh (shouldn't be needed)
# !apt-get install -y libglu1-mesa
# !pip install pygmsh gmsh

In [ ]:
# Import libraries
from firedrake import *
import numpy as np
import matplotlib.pyplot as plt
import firedrake.pyplot as fdplt
from time import perf_counter # Import the tool to compute the iteration time
from scipy.linalg import null_space
import math

# *NOTE: (on NON-homogeneous Dirichlet)*
For NON-homogeneous Dirichlet in theory you should use the LIFTING of the boundary, and the code will be slightly different, you still assume homogeneous Diriclet and then set the lifting $R$, the code will change only at the end as follow:

In [ ]:
# Define the boundary condition (in our case homogeneous Dirichlet)
bc = DirichletBC(V, Constant(0.), "on_boundary") # on_boundary is a keyword to set boundary conditions on all boundaries


# Define the variational problem
a = dot(grad(u), grad(v)) * dx # Bilinear form
L = f*v*dx # Linear function


# Solve the problem:
u_tilde = Function(V) # Define the solution as belonging to the space V
solve(a==L, u_tilde, bcs=bc)


# Reconstruct using the lifting
R = 1 - x # Lifting of the boundary condition
u_h = project(u_tilde + R, V) # We need to interpolate because the two objects are defined in different ways

Firedrake can take care of the lifting for us. We just need to specify the non-homogeneous boundary conditions. The procedure is then to build a lifting that belongs to the space of trial functions and that is equal to the boundary condition on boundary nodes and zero in all other mesh nodes.

# **ELLIPTIC PROBLEM**

\begin{align}
    \begin{cases}
       - ∇\cdotp (K∇u) = f \quad \quad \quad \quad \quad \quad \quad ~~~ in~Ω \\
        u = g_D \quad \quad \quad ~~ \quad \quad \quad \quad \quad \quad \quad \quad on~Γ_D\\
        -K ∇u· n = g_N \quad  \quad \quad \quad \quad \quad \quad \quad on~\Gamma_N \\
        -K ∇u·n = α(u-u_R) \quad \quad \quad \quad \quad on~\Gamma_R \\
    \end{cases}
\end{align}

Where $K\in \mathbb{R}^{2×2}$.

In [ ]:
#------- Initialize problem data -------
alpha = Constant(1.)
f = Constant(0.)
u_ex = lambda x, y: np.sin(pi*x) * np.sin(pi*y) # If this then you recall it with u_ex(x, y) (is a python-function)
# u_ex = sin(pi*x) * sin(pi*y) # If this then you recall it with u_ex (is a firedrake-function)
                               # N.B --> !!!!!!!!! this has to be defined after x, y = SpatialCoordinate(mesh) !!!!!!!!!
# B.C.s:
gD = Constant(0.) # Homogeneous Diriclét B.C.
gN = Constant(-1.) # Generic Neuman B.C.
uR = Constant(1.) # Generic Robin B.C.


#------- Define the mesh -------
N = 20# Number of mesh elements
# mesh = UnitIntervalMesh(N) # Mesh in (0, 1)
mesh = UnitSquareMesh(N, N) # Firedrake comand to make a squared mesh in (0, 1)x(0, 1)
# mesh = UnitSquareMesh(N, N, diagonal='crossed')
x, y = SpatialCoordinate(mesh) # Get mesh coordinates (if you want to evaluate some function of x, y as f(x,y))
# x = SpatialCoordinate(mesh) # This case you recall f as f(x[0],x[1])


# ------ Plot the mesh: -------
# fig, ax = plt.subplots()
# q = fdplt.triplot(mesh, axes=ax)
# ax.legend()
# ax.set_aspect('equal')



#------- Define the function space: -------
V = FunctionSpace(mesh, 'P', 1) # P^1 elements


#------- Define trial and test functions as belonging to the space: -------
u = TrialFunction(V) # Trial function
v = TestFunction(V) # Test function


#------- Enforce Diriclet B.C.: -------
bc = DirichletBC(V, gD, (1, 2)) # If only in boundary (1, 2)
# bc = DirichletBC(V, gD, "on_boundary") # If in all the boundary
# bc = (bc_1, bc_2, ...) if you have different Dirichlet in different boundaries


#------- Define the variational problem: -------
a = dot(grad(u), grad(v))*dx + alpha*u*v*ds(4) # Bilinear form
L = f*v*dx - gN*v*ds(3) + alpha*uR*v*ds(4) # Linear function
# Note:
# dx --> firedrake command for integration in the hole domain
# ds(i) --> firedrake command for integration in the i-th boundary


#------- Get the approximate solution: -------
u_h = Function(V) # Initialize the solution
solve(a==L, u_h, bcs=bc)


#------- Get the errors on H1 and L2: -------
err_H1 = errornorm(u_ex(x, y), u_h, 'H1')
err_L2 = errornorm(u_ex(x, y), u_h, 'L2')



#------- Plot the solution (use firedrake): -------
fig, ax = plt.subplots()
q = fdplt.tripcolor(u_h, axes=ax)
plt.colorbar(q)

# **PARABOLIC PROBLEM**

On the unit square domain $\Omega = (0,1) \times (0,1)$ and time interval $I = (0, 1)$, we have: 

  $$
    \begin{cases}
      \dfrac{\partial u}{\partial t}- \Delta u = f  & \text{in } \Omega\times I, \\
      u = g_D                                         & \text{on } \partial\Omega\times I, \\
      {\left. u \right|}_{t=0} = u_0                & \text{on } \Omega, \\
    \end{cases}
$$

We need to get a method to discretize in time, and this is given by the **$\theta$-method**.


**Semi-discrete in time weak form**
For the sake of implementation in Firedrake, we need the following formulation to build the needed forms:

$$
\underbrace{\left(\frac{u_{n+1}}{\Delta t},v \right)_\Omega
+\theta a(u_{n+1},v)}_{\tilde a(u_{n+1}, v)} =
\underbrace{\left(\frac{u_{n}}{\Delta t},v \right)_\Omega - (1-\theta) a(u_n,v) + \left(\theta f_{n+1} + (1-\theta) f_n,v\right)_\Omega}_{L(v)}
$$

So the Weak formulation becomes:

$$
\tilde a(u_{n+1}, v)=L(v)
$$

In [ ]:
#------- Initialize problem data -------
alpha = Constant(1.)
u_ex = lambda x, y: np.sin(pi*x) * np.sin(pi*y) # If this then you recall it with u_ex(x, y) (is a python-function)
# u_ex = sin(pi*x) * sin(pi*y) # If this then you recall it with u_ex (is a firedrake-function)
                               # N.B --> !!!!!!!!! this has to be defined after x, y = SpatialCoordinate(mesh) !!!!!!!!!
# B.C.s:
gD = Constant(0.) # Homogeneous Diriclét B.C.
gN = Constant(-1.) # Generic Neuman B.C.
uR = Constant(1.) # Generic Robin B.C.
T = 1. # Time domain






#------- Define the mesh -------
N = 20# Number of mesh elements
# mesh = UnitIntervalMesh(N) # Mesh in (0, 1)
# mesh = UnitSquareMesh(N, N) # Firedrake comand to make a squared mesh in (0, 1)x(0, 1)
mesh = UnitSquareMesh(N, N, diagonal='crossed')
x, y = SpatialCoordinate(mesh) # Get mesh coordinates (if you want to evaluate some function of x, y as f(x,y))
# x = SpatialCoordinate(mesh) # This case you recall f as f(x[0],x[1])



# ------ Plot the mesh: -------
# fig, ax = plt.subplots()
# q = fdplt.triplot(mesh, axes=ax)
# ax.legend()
# ax.set_aspect('equal')



#------- Define the function space: -------
V = FunctionSpace(mesh, 'P', 1) # P^1 elements


#------- Define trial and test functions as belonging to the space: -------
u = TrialFunction(V) # Trial function
v = TestFunction(V) # Test function


#------- Enforce Diriclet B.C.: -------
# bc = DirichletBC(V, gD, (1, 2)) # If only in boundary (1, 2)
bc = DirichletBC(V, gD, "on_boundary") # If in all the boundary
# bc = (bc_1, bc_2, ...) if you have different Dirichlet in different boundaries



#------- Define trial and test functions as belonging to the space: -------
u = TrialFunction(V) # Trial function
v = TestFunction(V) # Test function


#------- Set initial quantities of time and solution -------
t = 0 # Set initial time
dt = 0.1 # Time-step
#
u_ex = lambda t: 1 + 0.5 * sin(pi*t) * sin(pi*x) * sin(pi*y) # Exact solution
f_fun = lambda t: (pi/2*cos(pi*t)) * sin(pi*x) * sin(pi*y) # We need to make f an explicit function of t, since we need to vary t at each step
#
u_old = Function(V)
u_old.assign(project(u_ex(t), V)) # We need to project because (same reason as lab 2&3)
#
u_h = Function(V, name='Temperature') # We give a name to the solution
u_h.assign(u_old)
#
errL2 = 0.
errH1 = 0.
# start_time = perf_counter() # Compute the time needed to solve the problem


#------- Iterative procedure to solve the problem: -------
while t<T:

    #------- Update time: -------
    t_old = t # Update previous t
    t += dt # Define the new t
    #print('t = ', t)

    #------- Build the problem: -------
    a = 1/dt * u * v * dx + theta * dot(grad(u), grad(v)) * dx
    # THETA-method
    L = ( 1/dt * u_old * v * dx - (1-theta) * (dot(grad(u_old), grad(v)) * dx)
            + theta * f_fun(t) * v * dx + (1-theta) * (f_fun(t_old) * v * dx) )
    solve(a==L, u_h, bcs=bc)
    u_old = u_h

    #------- Compute the error: -------
    # max over t is = ||u_h(t)-u_ex(t)||_H^1 <= C_1 delta t^p + C_2 h^r
    # If u_ex belongs to H^(r+1) and u_h belongs to P^r
    # P = {2 if zita = 0.5
    #     {1 all the other case
    errL2 = max(errL2, errornorm(u_ex(t), u_h, 'L2'))
    errH1 = max(errH1, errornorm(u_ex(t), u_h, 'H1'))

#exact_time = perf_counter() - start_time


#------- Plot the solution: -------
fig, ax = plt.subplots()
q = fdplt.tripcolor(u_h, axes=ax)
plt.colorbar(q)

# **ADVECTION-DIFFUSION**

Consider the domain $Ω = (0, 1) × (0, 1)$, the ADVECTION-DIFFUSION problem is:

$$
  \begin{cases}
    -μΔu + \boldsymbol{b}\cdot∇u = f \quad in \, Ω \\
    u = g_{D} \quad \quad \quad \quad \quad \quad on \, ∂Ω
  \end{cases}
$$

Here we need stabilization methods if $\mathbb{P}e>1$, therefore we can adopt *STREAMLINE DIFFUSION* or *ARTIFICIAL DIFFUSION* but they are NOT storngly consistent, so the STRONGLY CONSTISTENT STABILIZING methods are *STREAMLINE UPWIND PETROV-GALERKIN* (SUPG) and *GALERKIN LEAST SQUARES* (GLS). We see the implemented method plus the stabilized solution for **SUPG** and **GLS** in the following code.

In [ ]:
#------- Initialize problem data -------
mu = Constant(1e-2)
b = Constant(as_vector((-1, -1)))
f = Constant(0.)
f = Constant(0.)
u_ex = lambda x, y: (np.exp(-x/mu) + np.exp(-y/mu) - 2)/(np.exp(-1/mu) - 1) # If this then you recall it with u_ex(x, y) (is a python-function)
# u_ex = (exp(-x/mu) + exp(-y/mu) - 2)/(exp(-1/mu) - 1) # If this then you recall it with u_ex (is a firedrake-function)
                                                        # N.B --> !!!!!!!!! this has to be defined after x, y = SpatialCoordinate(mesh) !!!!!!!!!
# B.C.s:
gD = Constant(0.) # Homogeneous Diriclét B.C.
gN = Constant(-1.) # Generic Neuman B.C.
uR = Constant(1.) # Generic Robin B.C.



#------- Define the mesh -------
N = 20# Number of mesh elements
# mesh = UnitIntervalMesh(N) # Mesh in (0, 1)
mesh = UnitSquareMesh(N, N) # Firedrake comand to make a squared mesh in (0, 1)x(0, 1)
# mesh = UnitSquareMesh(N, N, diagonal='crossed')
x, y = SpatialCoordinate(mesh) # Get mesh coordinates (if you want to evaluate some function of x, y as f(x,y))
# x = SpatialCoordinate(mesh) # This case you recall f as f(x[0],x[1])


# ------ Plot the mesh: -------
# fig, ax = plt.subplots()
# q = fdplt.triplot(mesh, axes=ax)
# ax.legend()
# ax.set_aspect('equal')



#------- Define the function space: -------
V = FunctionSpace(mesh, 'P', 1) # P^1 elements


#------- Define trial and test functions as belonging to the space: -------
u = TrialFunction(V) # Trial function
v = TestFunction(V) # Test function


#------- Enforce Diriclet B.C.: -------
bc = DirichletBC(V, gD, (1, 2)) # If only in boundary (1, 2)
# bc = DirichletBC(V, gD, "on_boundary") # If in all the boundary
# bc = (bc_1, bc_2, ...) if you have different Dirichlet in different boundaries



#------- Define the variational problem: -------
a = mu * dot(grad(u), grad(v)) * dx + dot(b, grad(u)) * v * dx # Bilinear form
L = f * v * dx # Linear function
# Note:
# dx --> firedrake command for integration in the hole domain
# ds(i) --> firedrake command for integration in the i-th boundary



# ---------------------------- STABILIZATION: ----------------------------
beta = 1 # Knob to tune the stabilization (remember beta>=0)
b_norm = sqrt(dot(b, b))
h = CellDiameter(mesh) # Function of firedrake to get h
Pe = b_norm*h/(2*mu)
tau = beta*h/(2*b_norm)*conditional(Pe/3>1, 1, Pe/3)

# a_s = tau * dot(b, grad(u)) * dot(b, grad(v)) * dx # Stabilization a


# -------------- STRONGLY CONSISTENT methods: --------------
rho = 0 # Method SUPG
# rho = 1 # Method GLS
R = -mu*div(grad(u))+dot(b, grad(u))
P = rho*(-mu*div(grad(v))) + (dot(b, grad(v)))

a_s = tau * R * P * dx
# ----------------------------------------------------------
# -----------------------------------------------------------------------



#------- Get the approximate solution: -------
u_h = Function(V) # Initialize the solution
# solve(a==L, u_h, bcs=bc) # For unstable solution
solve(a+a_s==L, u_h, bcs=bc) # For stable solution


#------- Get the errors on H1 and L2: -------
err_H1 = errornorm(u_ex(x, y), u_h, 'H1')
err_L2 = errornorm(u_ex(x, y), u_h, 'L2')



#------- Plot the solution (use firedrake): -------
fig, ax = plt.subplots()
q = fdplt.tripcolor(u_h, axes=ax)
fdplt.triplot(mesh, axes=ax)
plt.colorbar(q)

In [ ]:
#------- Peclét number: -------
h = (1-0)/N
b = np.array([-1,-1])
mu = 10**(-2)
Pe = (np.linalg.norm(b, ord=1)*h)/(2*mu)
print(f'The Peclet number is: Pe = ',Pe)

# **DIFFUSION-REACTION**

Consider the PDE problem 

$$
\begin{cases}
-\mu ∆u + \sigma u = f \quad \quad \text{in} \, \, Ω = (0, 1)\text{x}(0, 1) \\
u = 0 \quad \quad \quad \quad \quad \quad \, \, \, \text{on} \, ∂Ω
\end{cases}
$$ \\

Here you need a stabilization procedure based on the LUMPING of the MASS MATRIX $\underline{\underline{M}}$ and you can either do it by hand summing up the rows and diagonalize it or using a *quadrature formula* which is what is implemented in the following code.

In [ ]:
#------- Initialize problem data -------
mu = Constant(1e-4)
sigma = Constant(1.)
f = Constant(0.)
u_ex = lambda x, y: np.sin(pi*x) * np.sin(pi*y) # If this then you recall it with u_ex(x, y) (is a python-function)
# u_ex = sin(pi*x) * sin(pi*y) # If this then you recall it with u_ex (is a firedrake-function)
                               # N.B --> !!!!!!!!! this has to be defined after x, y = SpatialCoordinate(mesh) !!!!!!!!!
# B.C.s:
gD = Constant(0.) # Homogeneous Diriclét B.C.
gN = Constant(-1.) # Generic Neuman B.C.
uR = Constant(1.) # Generic Robin B.C.



#------- Define the mesh -------
N = 20# Number of mesh elements
# mesh = UnitIntervalMesh(N) # Mesh in (0, 1)
mesh = UnitSquareMesh(N, N) # Firedrake comand to make a squared mesh in (0, 1)x(0, 1)
# mesh = UnitSquareMesh(N, N, diagonal='crossed')
x, y = SpatialCoordinate(mesh) # Get mesh coordinates (if you want to evaluate some function of x, y as f(x,y))
# x = SpatialCoordinate(mesh) # This case you recall f as f(x[0],x[1])


# ------ Plot the mesh: -------
# fig, ax = plt.subplots()
# q = fdplt.triplot(mesh, axes=ax)
# ax.legend()
# ax.set_aspect('equal')



#------- Define the function space: -------
V = FunctionSpace(mesh, 'P', 1) # P^1 elements


#------- Define trial and test functions as belonging to the space: -------
u = TrialFunction(V) # Trial function
v = TestFunction(V) # Test function


#------- Enforce Diriclet B.C.: -------
bc = DirichletBC(V, gD, (1, 2)) # If only in boundary (1, 2)
# bc = DirichletBC(V, gD, "on_boundary") # If in all the boundary
# bc = (bc_1, bc_2, ...) if you have different Dirichlet in different boundaries



# Define the weak formulation:
u = TrialFunction(V)
v = TestFunction(V)


#------- Define the variational problem: -------
#-----------STABILIZATION with LUMPED-MASS MATRIX -----------
quad_rule = finat.quadrature.make_quadrature(V.finat_element.cell, 1, "KMV")
dxlump=dx(scheme=quad_rule) # Lumped integration rule
# m_lump = u*v*dxlump
#------------------------------------------------------------
a = mu*dot(grad(u), grad(v)) * dx + sigma * u * v * dxlump
L = f * v * dx



#------- Get the approximate solution: -------
u_h = Function(V)
solve(a==L, u_h, bcs=bc)


#------- Get the errors on H1 and L2: -------
err_H1 = errornorm(u_ex(x, y), u_h, 'H1')
err_L2 = errornorm(u_ex(x, y), u_h, 'L2')


# Plot the results:
fig, ax = plt.subplots()
q = fdplt.tricontourf(u_h, axes=ax)
plt.colorbar(q)

In [ ]:
h = CellDiameter(mesh)
Pe = sigma*h**2/(6*mu) # Pointwise value
meanPe = assemble(Pe*dx)
print('Mean Peclet number = ', meanPe)

# **STOKES PROBLEM**

$$
  \begin{cases}
    - \Delta \boldsymbol{u}+\boldsymbol{∇}p = \boldsymbol{f} \quad \quad \text{in} \, Ω \\
    \boldsymbol{∇}⋅\boldsymbol{u}=0  \quad \quad \quad \quad \quad \text{in} \, Ω \\
    \boldsymbol{u} = \boldsymbol{g}_D \quad \quad \quad \quad \quad \text{on} \, \partialΩ
  \end{cases}
$$

Here the problem is the stabilization, you can either use adequate $\mathbb{P}_2-\mathbb{P}_1$ elements or use $\mathbb{P}_1-\mathbb{P}_1$ elements and a stabilization procedure for pressure. The stabilization procedure is shown in the following code, if you instead choose $\mathbb{P}_2-\mathbb{P}_1$ elements, then you don't need it.

In [ ]:
#------- Initialize problem data -------
# B.C.s:
gD = Constant(0.) # Homogeneous Diriclét B.C.
gN = Constant(-1.) # Generic Neuman B.C.
uR = Constant(1.) # Generic Robin B.C.



#------- Define the mesh -------
N = 20# Number of mesh elements
# mesh = UnitIntervalMesh(N) # Mesh in (0, 1)
# mesh = UnitSquareMesh(N, N) # Firedrake comand to make a squared mesh in (0, 1)x(0, 1)
mesh = UnitSquareMesh(N, N, diagonal='crossed')
x, y = SpatialCoordinate(mesh) # Get mesh coordinates (if you want to evaluate some function of x, y as f(x,y))
# x = SpatialCoordinate(mesh) # This case you recall f as f(x[0],x[1])


# ------ Plot the mesh: -------
# fig, ax = plt.subplots()
# q = fdplt.triplot(mesh, axes=ax)
# ax.legend()
# ax.set_aspect('equal')



#------- Define the function space: -------
V = VectorFunctionSpace(mesh, 'P', 1) # Function space of u (Unstable)
Q = FunctionSpace(mesh, 'P', 1) # Function space of q
# if P0 --> Q = FunctionSpace(mesh, 'DP', 0) #P0 element
W = V*Q # Function space of the output
# W = MixedFunctionSpace([V, Q.])


#------- Define trial and test functions as belonging to the space: -------
u, p = TrialFunctions(W)
v, q = TestFunctions(W)



#------- Enforce Diriclet B.C. and Pressure initial value: -------
bcs = [DirichletBC(W.sub(0), Constant((1., 0.)), 4),
       DirichletBC(W.sub(0), Constant((0., 0.)), (1, 2, 3))]
# Handle the pressure indetermination by specifying the nullspace
# see https://www.firedrakeproject.org/solving-interface.html#singular-operators-in-mixed-spaces
# Following the documentation, we provide a basis for the nullspace of pressure
# For velocity, we pass the whole space, signifying that "we don't care about" it
nullspace = MixedVectorSpaceBasis(
    W, [W.sub(0), VectorSpaceBasis(constant=True)])





#------- Define the variational problem: -------
# N.B. --> Here you have to use inner instead of dot because you have W as
#          multi-dimensional space:
a = inner(grad(u), grad(v))*dx - p*div(v)*dx - q*div(u)*dx
L = dot(f, v) * dx
#------------------- Stabilizatin term for pressure: -------------------
delta = Constant(1.)
h = CellDiameter(mesh)
a_stab = -delta * h * h * dot(grad(p), grad(q)) * dx
#-----------------------------------------------------------------------



#------- Get the approximate solution: -------
w = Function(W)
# solve(a==L, w, bcs, nullspace=nullspace) # Unstable
solve(a+a_stab==L, w, bcs, nullspace=nullspace) # Stable
u_h, p_h = w.subfunctions # Sice we have two solution in this way we extrac them
                          # one for velocity u_h and one for pressure p_h



#------- Get the errors on H1 and L2: -------
err_H1 = errornorm(u_ex(x, y), u_h, 'H1')
err_L2 = errornorm(u_ex(x, y), u_h, 'L2')





#------- Plot the solution (use firedrake): -------
fig, ax = plt.subplots(1,2)
# Plot speed:
quiver(u_h, axes=ax[0])
ax[0].set_aspect('equal')
ax[0].set_title('Speed solution')
# Plot pressure:
tripcolor(p_h, axes=ax[1])
ax[1].set_aspect('equal')
ax[1].set_title('Pressure solution')

# *ORDER OF CONVERGENCE plot:*

In [ ]:
# Define array of polynomial orders
P = np.array((1, 2, 3, 4, 5, 6, 7))

# Initialize arry of errors
err_H1_vec = []
err_L2_vec = []

for ii in range(len(P)):
  u_h, err_H1, err_L2 = solve_FE_recorsive(N, int(P[ii]))
  err_H1_vec.append(err_H1)
  err_L2_vec.append(err_L2)


#------- Plot errors (using matplotlib): -------
fig, ax = plt.subplots()
ax.loglog(P, err_H1_vec, '*', label='FEM') # Error H1
# Lines with different orders:
ax.loglog(P, 1e-1*1/P, '--', label='$Order 1$')
ax.loglog(P, 1e-1*1/P**2, '--', label='$Order 2$')
ax.loglog(P, 1e-1*1/P**3, '--', label='$Order 3$')
ax.loglog(P, 1e-1*1/P**4, '--', label='$Order 4$')
ax.grid(which='minor')
ax.set_title('$H^1$ error')
ax.legend()

fig, ax = plt.subplots()
ax.loglog(P, err_L2_vec, '*', label='FEM') # Error H1
# Lines with different orders:
ax.loglog(P, 1e-2*1/P**2, '--', label='$P^{-2}$')
ax.grid(which='minor')
ax.set_title('$L^2$ error')
ax.legend()